In [30]:
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [39]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [40]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.3,  # Increase shift range
    height_shift_range=0.3,
    shear_range=0.3,  # Increase shear range
    zoom_range=0.3,  # Increase zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Add vertical flip
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [41]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1211 images belonging to 4 classes.
Found 98 images belonging to 4 classes.


In [62]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.6),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [43]:
%pip install scipy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [71]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=250,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 349ms/step - accuracy: 0.8386 - loss: 0.4751 - val_accuracy: 0.7604 - val_loss: 0.9184
Epoch 2/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9062 - loss: 0.3366 - val_accuracy: 0.7500 - val_loss: 0.9439
Epoch 3/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 332ms/step - accuracy: 0.8376 - loss: 0.4204 - val_accuracy: 0.7292 - val_loss: 0.8141
Epoch 4/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9062 - loss: 0.4576 - val_accuracy: 0.7396 - val_loss: 0.7483
Epoch 5/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 329ms/step - accuracy: 0.8481 - loss: 0.4146 - val_accuracy: 0.7396 - val_loss: 0.9748
Epoch 6/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8438 - loss: 0.4016 - val_accuracy: 0.7292 - val_loss: 1.0338
Epoch 7/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 342ms/step - accuracy: 0.8426 - loss: 0.4570 - val_accuracy: 0.7292 - val_loss: 0.9579
Epoch 8/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7812 - loss: 0.6393 - val_accu

In [72]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8079 - loss: 1.0171 
Test accuracy: 0.8061224222183228


In [80]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data\test\download.jpeg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
The snake species is: Bungarus multicinctus
